# Yet Another OpenPose 

---
## Environment

In [1]:
#verify user
!whoami
#verify user account, if running on google cloud, otherwise ignore
!gcloud auth list
#which Environment/virtualenv running in
import sys
import os

print("Python running from:",sys.prefix)
print("Current working dir",os.getcwd())

desktop-2qv6e1h\flash
      Credentialed Accounts
ACTIVE  ACCOUNT
*       michael.zl.prime@gmail.com
Python running from: c:\program files\python36-64
Current working dir C:\Users\flash\Project\Yet-Another-Openpose-Implmentation



To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [2]:
#start tensor board
# must run 
#/usr/local/bin/tensorboard serve --logdir gs://dl_training_results/tensorboard --port 8889 --bind_all

In [3]:
#start TPU
#!gcloud compute tpus start node-1 --zone us-central1-c
#stop TPU
#!gcloud compute tpus stop node-1 --zone us-central1-c

---
## Config

In [4]:
import default_config as cfg

import tpu_training.TPU_config as TPU_config
#cfg.update(TPU_config) #comment out to disable TPU training
cfg.RUN_NAME="Should_work_now" #for reference 

In [5]:
import tensorflow as tf
print("Tensorflow version:",tf.version.VERSION)

Tensorflow version: 2.0.0


In [6]:
if cfg.TPU_MODE:
    print("Testing results bucket connectivity")
    !touch /tmp/test
    !gsutil cp /tmp/test {cfg.TENSORBOARD_PATH}/test
    !gsutil rm {cfg.TENSORBOARD_PATH}/test
    !gsutil cp /tmp/test {cfg.CHECKPOINTS_PATH}/test
    !gsutil rm {cfg.CHECKPOINTS_PATH}/test
    print("Testing dataset bucket connectivity")
    !gsutil ls gs://{cfg.GCS_TFRECORDS_BUCKETNAME} | head -4
    print("Testing TPU connectivity")
    !nmap -Pn -p8470 {cfg.TPU_IP}
    import tpu_training.init_tpu as init_tpu
    strategy,resolver=init_tpu.init_tpu() #This must be run before any imports!!!!

In [7]:
import datetime
import dataset_functions
import models.six_stage_linear_model as model
import callbacks
import dataset_builder
import load_weights
import loss_metrics
from utils import now

In [8]:
# import importlib as il
# il.reload() #useful to reload any changes

---
## Dataset

In [9]:
tfrecord_files_train=dataset_builder.get_tfrecord_filenames(cfg.TRAIN_TFRECORDS,cfg)
tfrecord_files_valid=dataset_builder.get_tfrecord_filenames(cfg.VALID_TFRECORDS,cfg)
print("Found the following training TFrecords:\n","\n".join(tfrecord_files_train))
print("Found the following validation TFrecords:\n","\n".join(tfrecord_files_valid))

print("Building training dataset")
dst=dataset_builder.build_training_ds(tfrecord_files_train,model.place_training_labels,cfg)
print("Training dataset shape:",dst)
print("Building validation dataset")
dsv=dataset_builder.build_validation_ds(tfrecord_files_valid,model.place_training_labels,cfg)
print("Validation dataset shape:",dsv)

Retrieving TFrecords from: .\dataset\TFrecords\training
Retrieving TFrecords from: .\dataset\TFrecords\validation
Found the following training TFrecords:
 .\dataset\TFrecords\training-001.tfrecords
.\dataset\TFrecords\training-002.tfrecords
Found the following validation TFrecords:
 .\dataset\TFrecords\validation-001.tfrecords
Building training dataset
Training dataset shape: <PrefetchDataset shapes: (((None, 368, 368, 3), (None, 46, 46, 1)), ((None, 46, 46, 35), (None, 46, 46, 35), (None, 46, 46, 35), (None, 46, 46, 35), (None, 46, 46, 19), (None, 46, 46, 19))), types: ((tf.float32, tf.float32), (tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32))>
Building validation dataset
Validation dataset shape: <BatchDataset shapes: (((None, 368, 368, 3), (None, 46, 46, 1)), ((None, 46, 46, 35), (None, 46, 46, 35), (None, 46, 46, 35), (None, 46, 46, 35), (None, 46, 46, 19), (None, 46, 46, 19))), types: ((tf.float32, tf.float32), (tf.float32, tf.float32, tf.float32, tf.float

In [10]:
#Test element
# dst_iter=iter(dst)
# sample_elem=next(dst_iter)
# print("Dataset shape:",dst) #this should match the model input, and output stages

In [11]:
#testing mask
# sample_elem=next(dst_iter)
# m=sample_elem[1][1][0,...,0]
# import matplotlib.pyplot as plt
# plt.imshow(m)
# plt.colorbar()

---
## Model
### Load weights

In [14]:
if cfg.ASK_FOR_CHECKPOINTS:
    checkpoint,starting_epoch=load_weights.checkpoints_prompt(cfg)
else:
    checkpoint=None
    starting_epoch=0

Found these checkpoints
0.Dont load checkpoint


### Build Model

In [15]:
model_maker=model.ModelMaker() #must be outside scope to keep the graph clean
tf.keras.backend.clear_session() #to clean to backaend from the imported model
def define():
    train_model,test_model=model_maker.create_models()
    
#     if cfg.INCLUDE_MASK:
#         losses=[loss_metrics.MaskedMeanAbsoluteError()
#                 ,loss_metrics.MaskedMeanAbsoluteError()
#                 ,loss_metrics.MaskedMeanAbsoluteError()
#                 ,loss_metrics.MaskedMeanAbsoluteError()
#                 ,loss_metrics.MaskedMeanSquaredError()
#                 ,loss_metrics.MaskedMeanSquaredError()]
                      
#     else:
#         raise NotImplementedError       
    
    #this must match the model output order
#     metrics=[
#           []
#          ,[]
#          ,[]
#          ,[]
#          ,[tf.keras.metrics.MeanAbsoluteError()]    
#          ,[tf.keras.metrics.MeanAbsoluteError()]
#         ]
    
    train_model.compile(
                    optimizer=tf.keras.optimizers.Adam(cfg.BASE_LEARNING_RATE)                   
                    ,loss=loss_metrics.MaskedMeanSquaredError()
                    ,loss_weights=[10,10,10,10,0.1,0.1]
                    #,metrics=metrics                           
                   )
    return train_model,test_model

if cfg.TPU_MODE:
    with strategy.scope():
        train_model,test_model=define()
        if (checkpoint):
            train_model.load_weights(checkpoint)
else:
    train_model,test_model=define()
    if (checkpoint):
        train_model.load_weights(checkpoint)

### Callbacks

In [16]:
all_callbacks=[
    callbacks.make_checkpoint_callback(cfg)
    ,callbacks.make_tensorboard_callback(cfg)
    ,callbacks.make_LRscheduler_callback(cfg.LEARNING_RATE_SCHEDUELE)
    ,callbacks.PrintLR()
    ,tf.keras.callbacks.TerminateOnNaN()
]


In [ ]:
#run to clean tensorboard results
#!gsutil -m rm -r {cfg.TENSORBOARD_PATH}/*
#!gsutil -m rm -r {cfg.CHECKPOINTS_PATH}/*

---
## Training
Actually training

In [17]:
train_history=train_model.fit(
    dst
    ,epochs=cfg.TRAINING_EPOCHS
    ,steps_per_epoch=cfg.STEPS_PER_EPOCH
    ,validation_data=dsv
    ,callbacks=all_callbacks
    ,initial_epoch=starting_epoch
)

Train for 28000 steps

Learning rate for epoch 0 is 0.0010000000474974513
Epoch 1/100
    1/28000 [..............................] - ETA: 451:19:29 - loss: 2.9915 - s1pafs_output_loss: 0.0684 - s2pafs_output_loss: 0.0480 - s3pafs_output_loss: 0.0363 - s4pafs_output_loss: 0.1460 - s5kpts_output_loss: 0.0209 - s6kpts_output_loss: 0.0172
Epoch 00001: saving model to /home/michael_zl_prime/training\checkpoints\Should_work_now11Wed1219-0013\-E0001.ckpt
    1/28000 [..............................] - ETA: 813:01:50 - loss: 2.9915 - s1pafs_output_loss: 0.0684 - s2pafs_output_loss: 0.0480 - s3pafs_output_loss: 0.0363 - s4pafs_output_loss: 0.1460 - s5kpts_output_loss: 0.0209 - s6kpts_output_loss: 0.0172

ProfilerNotRunningError: Cannot stop profiling. No profiler is running.

---
### Save results

In [ ]:
tmp_path='gs://dl_training_results/tmp/tensorflow/temp_weights'
train_model.save_weights(tmp_path)

cpu_train_model,cpu_test_model=define()

cpu_train_model.load_weights(tmp_path)
cpu_test_model.load_weights(tmp_path)

cpu_test_model.save(cfg.MODELS_PATH+"test_"+now()+cfg.RUN_NAME) 
cpu_train_model.save(cfg.MODELS_PATH+"train_"+now()+cfg.RUN_NAME) 

---
### Shutdown

In [ ]:
#shut down TPU
!gcloud compute tpus stop node-1 --zone us-central1-c

In [ ]:
!gcloud compute instances stop instance-1 --zone us-central1-c